# Régression & optimisation par descente de gradient

Ce tp a deux objectifs: 
 - Acquérir les connaissances de base pour faire face au problème de la régression, c'est à dire de l'estimation d'un score correpondant à une situation,
 - Comprendre la descente de gradient et ses variantes (stochastique, mini-batch, moment,...)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Chapitre 1: régression

A partir de donnée étiquetées $\{(x_i,y_i)\}_{i=1,\ldots,n}$, apprendre un modèle $f$ tel que $f(x)\approx y$

<img src="fig/reg_intro.png">


## A. Génération de données jouet & construction d'une solution analytique

### A.1 Génération de données jouet 

[tout le code est donné... A vous de le comprendre]

Dans un premier temps, générons des données jouets paramétriques:
- $N$: nombre de points à générer
  - $x \in [0, 1]$ tirage avec un simple rand() ou un linspace() -au choix-
  - Si vous optez pour un tirage aléatoire des abscisses, triez les points pour simplifier les traitements ultérieurs
- $y=ax+b+\epsilon, \epsilon \sim \mathcal N(0,\sigma^2)$
  - Rappel : en multipliant un tirage aléatoire selon une gaussienne centrée réduite par $\sigma$ on obtient le bruit décrit ci-dessus

Afin de travailler sur les bonnes pratiques, nous distinguerons un ensemble d'apprentissage et un ensemble de test.
Les deux sont tirés selon la même distribution. L'ensemble de test comptera -arbitrairement- 1000 points.

In [ ]:
def gen_data_lin(a, b, sig, N=500, Ntest=1000):
    X_train = np.sort(np.random.rand(N)) # sort optionnel, mais ça aide pour les plots
    X_test  = np.sort(np.random.rand(Ntest)) 
    Y_train = a*X_train + b + np.random.randn(N)*sig # a minima, bien comprendre les dimensions des vecteurs
    Y_test  = a*X_test + b + np.random.randn(Ntest)*sig
    return X_train, Y_train, X_test, Y_test

In [ ]:
# génération de données jouets:
a = 6.
b = -1.
N = 50
sig = .4 # écart type

X_train, y_train, X_test, y_test = gen_data_lin(a, b, sig, N)

plt.figure()
#plt.plot(X_test, y_test, 'r.')
plt.scatter(X_train, y_train, c='b')
plt.grid()


###  TODO  ###

### A.2. Formulation au sens des moindres carrés

Nous partons directement sur une écriture matricielle. Du coup, il est nécessaire de construire la matrice enrichie $X$:
    $$X = \begin{pmatrix}
                x_0 & 1\\
                \vdots & \vdots\\
                x_n & 1
                \end{pmatrix}
                 $$
Le code de la fonction d'enrichissement est donné ci-dessous.


L'idée de cet enrichissement est de simplifier le code suivant:
1. facilité de construction de la fonction linéaire:
$$ \hat Y = X \cdot w, \qquad X\in \mathbb R^{n\times 2}, \qquad w = \begin{pmatrix}a\\ b\end{pmatrix}
\in \mathbb R^{2}, \qquad \forall i,\  \hat y_i = a x_i + b$$
2. L'idée est de minimiser l'erreur au sens des moindres carrés $C$:
$$w^\star = \arg\min_w C(X,Y,w) = \arg\min_w \|X \cdot w - Y\|^2 $$
3. Trouver l'argmin = annuler la dérivée du gradient:
$$\arg\min_w \|X \cdot w - Y\|^2 \Leftrightarrow  \nabla_w C(X,Y,w) = 2 X^T (X \cdot w - Y) =0 \Leftrightarrow  X^T X \cdot w =  X^T Y $$ 
4. Le problème d'apprentissage revient alors à résoudre un système d'équations linéaires de la forme:
$$ A w = B $$
**Rappel des formules vues en cours/TD:**
$$ A=X^T X \in \mathbb R^{d\times d}$$
$$ B=X^T Y\in \mathbb R^{d} $$
Fonction de résolution: `np.linalg.solve(A,B)`
Vous devez obtenir la même solution que précédemment. 

In [ ]:
def make_mat_lin_biais(X): # fonctionne pour un vecteur unidimensionel X
    N = len(X)
    return np.hstack((X.reshape(N,1),np.ones((N,1))))

In [ ]:
Xe = make_mat_lin_biais(X_train)
# A compléter pour résoudre le problème d'apprentissage

print(w)

### A.3. Evaluation des modèles

Il est nécessaire d'avoir une évaluation **quantitative** (critère d'erreur MSE -Mean Squared Error-, MAPE -Mean Average Percentage Error-, ...). Sur les exemples simples, il est possible de tracer le résultat pour une analyse **qualitative**.

In [ ]:
# Evaluation quantitative

def erreur_mc(y, yhat):
    return ((y-yhat)**2).mean()

# estimation des sorties avec les paramètres que vous avez calculé
yhat_train = # a compléter
yhat_test  = # a compléter

print('Erreur moyenne au sens des moindres carrés (train):', erreur_mc(yhat_train, y_train))
print('Erreur moyenne au sens des moindres carrés (test):', erreur_mc(yhat_test, y_test))

In [ ]:
# Evaluation qualitative

plt.figure()
plt.scatter(X_test, y_test, c='r', alpha=0.3)
plt.scatter(X_test, yhat_test, c='g')
plt.plot(X_test, yhat_test, 'g--')


## B. Construction de modèles non linéaire

Dans le formalisme mis en place ci-dessus, il est très facile de passer à des modèles non linéaire, par exemple polynomiaux.

1. Générer des données polynomiales pour valider votre approche
2. A partir des données brutes $\begin{pmatrix}x_0& x_1& \ldots& x_n\end{pmatrix}$, construction d'une matrice enrichie
 $$X = \begin{pmatrix}
                x_0^2 & x_0 & 1\\
                \vdots & \vdots\\
                x_n^2  & x_n & 1
                \end{pmatrix}
                 $$

3. Tout est fait ! 
$$ \hat Y = X \cdot w, \qquad X\in \mathbb R^{n\times 3}, \qquad w = \begin{pmatrix}a\\ b\\ c\end{pmatrix}
\in \mathbb R^{2}, \qquad \forall i,\  \hat y_i = a x_i^2 + b x_i +c$$

En reprennant la formulation matricielle ci-dessus, on voit que naturellement $w=\begin{pmatrix}a\\ b\\ c\end{pmatrix}$ permet de définir les coefficients du polynome.

OBJECTIF: <BR>
**TRANSFORMER les données $\Rightarrow$ TRANSFORMATION automatique du modèle**

<p style="color:red"> ATTENTION </p> 

Dans la suite, ne pas confondre l'ancien jeu de données `X_train, y_train` et le nouveau `Xp_train, yp_train`. 

In [ ]:
# Tout le code est donné
def gen_data_poly2(a, b, c, sig, N=500, Ntest=1000):
    '''
    Tire N points X aléatoirement entre 0 et 1 et génère y = ax^2 + bx + c + eps
    eps ~ N(0, sig^2)
    '''
    X_train = np.sort(np.random.rand(N))
    X_test  = np.sort(np.random.rand(Ntest))
    y_train = a*X_train**2+b*X_train+c+np.random.randn(N)*sig
    y_test  = a*X_test**2 +b*X_test +c+np.random.randn(Ntest)*sig
    return X_train, y_train, X_test, y_test

Xp_train, yp_train, Xp_test, yp_test = gen_data_poly2(10, -10, 5, 0.1, N=100, Ntest=100)
plt.figure()
plt.plot(Xp_train, yp_train)

In [ ]:
# Enrichissement de la matrice X => pour construire un modèle polynomial
def make_mat_poly_biais(X): 
    # A compléter
    return Xe

Xe   = make_mat_poly_biais(Xp_train)
Xe_t = make_mat_poly_biais(Xp_test)
w    = # A compléter 

yhat   = # A compléter
yhat_t = # A compléter
print('Erreur moyenne au sens des moindres carrés (train):', erreur_mc(yhat, yp_train))
print('Erreur moyenne au sens des moindres carrés (test):', erreur_mc(yhat_t, yp_test))

plt.figure()
plt.plot(Xp_train, yp_train, 'b.')
plt.plot(Xp_train, yhat, 'r')

# Chapitre 2: Descente de gradient

Se déplacer itérativement dans l'espace des paramètres (idéalement vers une solution optimale)

<img src="fig/batch.png">



## C. Fonction de coût & optimisation par descente de gradient

### C.1. Algorithme itératif

Nous allons maintenant résoudre le problème de la régression par minimisation d'une fonction de coût:
$$ C = \sum_{i=1}^N (y_i - f(x_i))^2$$

Soit un problème avec des données $(x_i,y_i)_{i=1,\ldots,N}$, une fonction de décision/prédiction paramétrée par un vecteur $w$ et une fonction de cout à optimiser $C(w)$.
Notre but est de trouver les paramètres $w^\star$ minimisant la fonction de coût:
$$ w^\star = \arg\min_w C(w)$$

l'algorithme de la descente de gradient est le suivant (rappel):

 - $w_0 \leftarrow init$ par exemple : 0
 - boucle
     - $w_{t+1} \leftarrow w_{t} - \epsilon \nabla_w C(w_t)$

Compléter le squelette d'implémentation fourni ci-dessous:

----
<p style="color:red"> <strong>ATTENTION</strong> </p> 

Afin de pouvoir visualiser l'évolution des paramètres, nous allons travailler dans un espace à 2 paramètres, c'est à dire dans le cas de la **régression linéaire**: assurez-vous de bien travailler sur les données du premier exercice: `X_train, y_train`

----


In [ ]:
# pour travailler en matrice: (re)construction de la matrice contenant les X et un biais
Xe = make_mat_lin_biais(X_train) # dataset linéaire, transformation lineaire des données
wstar = # A compléter pour se rappeler du w optimal (code issu de la section précédente)

def descente_grad_mc(X, y, eps=1e-4, nIterations=100):
    w = np.zeros(X.shape[1]) # init à 0
    allw = [w] # stocker toutes les valeurs que va prendre le gradient pour pouvoir l'afficher ensuite
    for i in range(nIterations):
        # A COMPLETER => calcul du gradient vu en TD
        #
        allw.append(w) # stockage de toutes les valeurs intermédiaires pour analyse
    allw = np.array(allw)
    return w, allw # la dernière valeur (meilleure) + tout l'historique pour le plot
    
w, allw = descente_grad_mc(Xe, y_train, eps=1e-4, nIterations=200)

On s'intéresse ensuite à comprendre la descente de gradient dans l'espace des paramètres. Le code ci-dessous permet de tracer le cout pour un ensemble de paramètres (toutes les valeurs de paramètres prises par l'algorithmes au fil du temps).


In [ ]:
# tracer de l'espace des couts
def plot_parametres( allw, X, y, opti = [], ngrid = 20, style ='b+-'):
    '''
    Fonction de tracer d'un historique de coefficients
    ATTENTION: ca ne marche qu'en 2D (évidemment)
    Chaque w doit contenir 2 valeurs
    
    Il faut fournir les données (X,y) pour calculer le cout associé 
    à un jeu de paramètres w
    ATTENTION X = forme matricielle des données
    '''
    w_min = np.minimum(wstar, np.min(allw,0)) # trouver les bornes
    w_max = np.maximum(wstar, np.max(allw,0))
    w_min -= (w_max-w_min)*0.1
    w_max += (w_max-w_min)*0.1
    # faire une grille régulière avec tous les couples possibles entre le min et le max
    w1range = np.linspace(w_min[0], w_max[0], ngrid)
    w2range = np.linspace(w_min[1], w_max[1], ngrid)
    w1,w2 = np.meshgrid(w1range,w2range)
    #
    # calcul de tous les couts associés à tous les couples de paramètres
    cost = np.array([[np.log(((X @ np.array([w1i,w2j])-y)**2).sum()) for w1i in w1range] for w2j in w2range])
    #
    # plt.figure()
    plt.contour(w1, w2, cost)
    if len(opti) > 0:
        plt.scatter(wstar[0], wstar[1],c='r')
    plt.plot(allw[:,0],allw[:,1],style ,lw=2 )
    return


In [ ]:
# invocation de la méthode définie ci-dessus
plt.figure()
plot_parametres( allw, Xe, y_train, opti=wstar)
# plt.savefig('fig/grad_descente.png')

In [ ]:
###  TODO  ###

Vous devez obtenir un image de la forme :
![Descente de gradient](fig/grad_descente.png)

### C.2 Comportement du gradient

Tester différents jeux de paramètres pour mettre en évidence les phénomènes suivants:
 - Divergence du gradient
 - Convergence incomplète (trop lente ou pas assez d'itération)
 - Convergence idéale: pas de gradient suffisamment grand et nombre d'itérations bien choisi

 Vous devriez obtenir des situations comme:
 ![Descente de gradient](fig/cvg_grad.png)
 

> <span style="color:magenta"> Le réglage du pas de gradient (**=learning rate**) est une étape critique dans toutes les approches de type réseaux de neurones </span>

In [ ]:
# Vos tests sur les paramètres de la descente de gradient

### C.3 Détection & Implémentation de la convergence

Jusqu'ici, on s'est contenté de descendre le grandient sur $N$ iterations... Il faut maintenant détecter la convergence pour sortir au bon moment:

1. On garde une structure algorithmique en boucle `for`: il est important de garder un nombre d'itération max pour éviter les boucles infinies... On va juste ajouter un `if` + `break` pour sortir en cas de convergence.
2. On sort quand?
    - lorsque le gradient s'annule (ou se rapproche suffisamment de 0)
    - lorsque le coût n'évolue plus (ou peu)
    - **il y a donc une notion de seuil** $\Rightarrow$ et donc un paramètre supplémentaire

In [ ]:
# on garde la méthode précédente qui marche... Et on en fait une nouvelle avec le critère de convergence
def descente_grad_mc_cvg(X, y, eps=1e-4, cvg = 1e-3, nIterations=200):
    w = np.zeros(X.shape[1]) # init à 0
    allw = [w] # stocker toutes les valeurs que va prendre le gradient pour pouvoir l'afficher ensuite
    for i in range(nIterations):
        # Reprendre le code déjà fait
        #
        allw.append(w) # stockage de toutes les valeurs intermédiaires pour analyse
        if # trouver un critère de convgergence
            break

    allw = np.array(allw)
    return w, allw # la dernière valeur (meilleure) + tout l'historique pour le plot

In [ ]:
# test
w, allw = descente_grad_mc_cvg(Xe, y_train, eps=1e-4,cvg = 1e-3, nIterations=200)
# vérification graphique (c'est l'avantage de travailler en 2D)
plt.figure()
plot_parametres( allw, Xe, y_train, opti=wstar)

### C.4 Gradient stochastique (SGD)

Rappel de l'idée générale: calculer une direction approximative du gradient sur un seul individu. La direction est moins chère à calculer mais il faut en faire plus.

<span style="color:red"> ATTENTION :</span> pour que ça marche, il faut voir les points dans un ordre aléatoire (d'où le nom stochastique)

| Gradient batch | Gradient stochastique (SGD) |
|--- | --- |
| - $w_0 \leftarrow init$ par exemple : 0 <BR> - boucle (itérations/convergence) <BR>  $\qquad$ - $w_{t+1} \leftarrow w_{t} - \epsilon \nabla_w C(X,Y,w_t)$  | - $w_0 \leftarrow init$ par exemple : 0 <BR> - boucle (itérations/convergence) <BR>  $\qquad$ <span style="color:red"> - Tirage d'un index $i$ </span> <BR> $\qquad$ - $w_{t+1} \leftarrow w_{t} - \epsilon \nabla_w C(\mathbf x_i,y_i,w_t)$  |

1. Rappel des formules de gradient:

$$C(X,Y,w) = \|X \cdot w - Y\|^2 = \sum_{i=1}^n C(\mathbf x_i,y_i,w) = \sum_{i=1}^n (\mathbf x_i \cdot w - y_i)^2$$
$$\nabla_w C(X,Y,w) = 2 X^T (X \cdot w - Y)  \qquad  \nabla_w C(\mathbf x_i,y_i,w) = 2 \mathbf x_i^T (\mathbf x_i \cdot w - y_i) $$

<span style="color:red">Note: </span> Vous risquez d'avoir des problèmes avec les opérations matricielles en passant à la version stochastique. Il faut bien calculer les dimensions des différents éléments de la formule.

2. Impact sur la convergence

Vous allez obtenir un résultat de la forme:
<img src="fig/grad_stoch.png">

Evidemment, la convergence est plus difficile à détecter. L'idée est de faire le test toutes les $n$ itérations (=**epoch**), c'est à dire une fois qu'on a vu tous les points (statistiquement).


In [ ]:
# on garde la méthode précédente qui marche... Et on en fait une nouvelle avec le critère de convergence
def descente_grad_mc_stoch(X, y, eps=1e-4, cvg = 1e-3, nIterations=200):
    w = np.zeros(X.shape[1]) # init à 0
    allw = [w] # stocker toutes les valeurs que va prendre le gradient pour pouvoir l'afficher ensuite
    for i in range(nIterations):
        # tirage d'un indice i
        
        # MAJ w

        # stockage de w
        allw.append(w) # stockage de toutes les valeurs intermédiaires pour analyse
        # test de convergence (à faire en fin d'énoncé)

    allw = np.array(allw)
    return w, allw # la dernière valeur (meilleure) + tout l'historique pour le plot

In [ ]:
###  TODO  ###

### C.5 Gradient mini-batch

Il s'agit logiquement d'une version intermédiaire: on calcule le gradient sur un sous-ensemble de points tirés aléatoirement (c'est ce qui est fait dans les réseaux de neurones modernes). On introduit un nouveau paramètre: **batch-size**.

<span style="color:red">Très proche des questions précédentes: vous pouvez garder cet exercice pour plus tard</span> 

In [ ]:
# implémentation du gradient mini-batch


### C.6 Moment supplémentaire, inertie dans le gradient stochastique

Le blog de S. Ruder explique particulièrement bien les améliorations possibles sur les descentes de gradient.

https://ruder.io/optimizing-gradient-descent/

Prenons le premier exemple, pour ajouter de l'inertie dans la descente de gradient stochastique:

$v_t = \gamma v_{t-1} + (1-\gamma) \nabla_w C$ <BR>
$w_t = w_{t-1} - \epsilon v_t$

1. Comment initialiser $v$?
2. Quel impact sur la descente de gradient stochastique en fonction de $\gamma$? Y a-t-il un cas limite permettant de revenir à la descente de gradient stochastique classique?

<img src="fig/grad_stoch_mom.png">

In [ ]:
# Gradient stochastique à moment

###  TODO  ###

### C.7 [Bonus] Et si on affichait tout ça en 3D? Et en animation?

De la 3D et de l'animation. Les fonctions ne marchent pas toutes parfaitement mais le résultat est sympa !

<img src = "fig/animation.gif">

In [ ]:
from matplotlib import cm
import time

def plot_parametres_3d( allw, X, y, opti, figax,  ngrid = 20):
    '''
    Quick & Dirty...
    '''
    w_min = np.minimum(wstar, np.min(allw,0)) # trouver les bornes
    w_max = np.maximum(wstar, np.max(allw,0))
    w_min -= (w_max-w_min)*0.1 # ajouter 10%
    w_max += (w_max-w_min)*0.1
    # faire une grille régulière avec tous les couples possibles entre le min et le max
    w1range = np.linspace(w_min[0], w_max[0], ngrid)
    w2range = np.linspace(w_min[1], w_max[1], ngrid)
    w1,w2 = np.meshgrid(w1range,w2range)
    #
    # calcul de tous les couts associés à tous les couples de paramètres
    cost = np.array([[np.log(((X @ np.array([w1i,w2j])-y)**2).sum()) for w1i in w1range] for w2j in w2range])

    costw = np.array([np.log(((X @ w-y)**2).sum()) for w in allw] )
    costwstar = np.log(((X @ opti-y)**2).sum())
    
    #figax.plot_wireframe(w1, w2, cost,  cmap='viridis', rstride=1, cstride=1)
    Z = (cost-cost.min())/(cost.max()-cost.min())
    colors = cm.viridis(Z)
    rcount, ccount, _ = colors.shape
    surf = figax.plot_surface(w1, w2, cost,  rcount=rcount, ccount=ccount,
                       facecolors=colors, shade=False)
    surf.set_facecolor((0,0,0,0))
    # if len(opti) > 0:
    #     plt.scatter(wstar[0], wstar[1],c='r')
    figax.plot(allw[:,0],allw[:,1],costw,c='m')
    figax.scatter(opti[0],opti[1],costwstar,c='r', s=200)
    return

def plot_reg(X,y,w):
    yhat = X@w
    plt.scatter(X[:,0],y)
    plt.plot(X[:,0],yhat,'m')
    return

Xe = make_mat_lin_biais(X_train) # dataset linéaire, transformation lineaire des données
wstar = np.linalg.solve(Xe.T@Xe, Xe.T@y_train) # A compléter pour se rappeler du w optimal (code issu de la section précédente)

fig = plt.figure(figsize=[16,8])
#plt.subplot(1,2,1)
ax = fig.add_subplot(1,2,1,projection='3d')
w, allw = descente_grad_mc_stoch(Xe, y_train, eps=1e-1,  nIterations=200) # il faut disposer de la fonction !
plot_parametres_3d( allw, Xe, y_train, wstar, ax)

#plt.figure()
fig.add_subplot(1,2,2)
plot_reg(Xe,y_train,wstar)

In [ ]:
# Creation d'une animation = sauvegarde dans un répertoire + gif animé sur le web
w, allw = descente_grad_mc_stoch(Xe, y_train, eps=2e-1,  nIterations=200)
fig = plt.figure(figsize=[16,8])

for i,wi in enumerate(allw):
    if i<50:
            if i% 3 !=0:
              continue        
    elif i% 5 !=0:
        continue
    #plt.subplot(1,2,1)
    ax = fig.add_subplot(1,2,1,projection='3d')
    plot_parametres_3d( allw[:np.maximum(i,1)], Xe, y_train, wstar, ax)

    #plt.figure()
    fig.add_subplot(1,2,2)
    plot_reg(Xe,y_train,wi)
    fig.savefig("fig/anim/anim_"+str(i)+".png")
    fig.clear()

# Chapitre 3: Passage sur des données réelles

Après avoir étudié trois manières de faire face au problème de la régression, nous proposons d'étudier un cas réel: la prédiction de la consommation des voitures en fonction de leurs caractéristiques.

Dans le cas présent, nous allons baser la solution sur la résolution analytique du problème des moindres carrés (`np.linalg.solve(A,B)`), qui semble la mieux adaptée au problème qui nous intéresse.

Le jeu de données est issu des datasets UCI, un répertoire parmi les plus connus en machine learning. Les données **sont déjà téléchargées et présentes dans le tme** mais vous voulez plus d'informations:
https://archive.ics.uci.edu/ml/datasets/auto+mpg

![voiture](fig/Large9.jpg)

Après avoir importé les données (fonction fournie), vous construirez une solution optimale et l'évaluerez au sens des moindres carrés en apprentissage et en test.


In [ ]:
import pandas as pd
# Chargement des données
data = pd.read_csv('data/auto-mpg.data', delimiter='\s+', header=None) # comme np.loadtxt mais en plus robuste
# remplacement des données manquantes '?' => Nan pour travailler sur des nombres
data.iloc[:,[3]] = data.iloc[:,[3]].replace('?', None)
data.iloc[:,[3]] = data.iloc[:,[3]].astype(float)
# remplacement des valeurs manquantes par la moyenne
data.iloc[:,[3]] = data.iloc[:,[3]].fillna(data.iloc[:,[3]].mean())

print(data.head()) # visualiser ce qu'il y a dans les données

X = np.array(data.values[:,1:-2], dtype=np.float64)
y = np.array(data.values[:,0], dtype=np.float64)


In [ ]:
# separartion app/test
def separation_train_test(X, y, pc_train=0.75):
    # A compléter
    # 1) générer tous les index entre 0 et N-1
    # 2) mélanger la liste
    napp = int(len(y)*pc_train) # nb de points pour le train
    X_train, y_train = X[index[:napp]], y[index[:napp]]
    X_test, y_test   = X[index[napp:]], y[index[napp:]]
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = separation_train_test(X, y, pc_train=0.75)

In [ ]:
# Résolution analytique

w = # A compléter
yhat   = # A compléter
yhat_t = # A compléter
print('Erreur moyenne au sens des moindres carrés (train):', erreur_mc(yhat, y_train))
print('Erreur moyenne au sens des moindres carrés (test):', erreur_mc(yhat_t, y_test))


In [ ]:
def plot_y(y_train, y_test, yhat, yhat_t):
    # tracé des prédictions:
    plt.figure()
    plt.subplot(211)
    plt.plot(y_test, label="GT")
    plt.plot(yhat_t, label="pred")
    plt.title('En test')
    plt.legend()
    plt.subplot(212)
    plt.plot(y_train, label="GT")
    plt.plot(yhat, label="pred")
    plt.title('En train')
    return
plot_y(y_train, y_test, yhat, yhat_t)

In [ ]:
# interprétation des poids
plt.figure()
plt.bar(np.arange(len(w)), w)

# Questions d'ouverture

## Sélection de caractéristiques

Quels sont les résultats obtenus en éliminant toutes les variables servent moins?

## Feature engineering

En étudiant la signification des variables du problèmes, on trouve:

1. mpg: continuous 
2. cylinders: multi-valued discrete 
3. displacement: continuous 
4. horsepower: continuous 
5. weight: continuous 
6. acceleration: continuous 
7. model year: multi-valued discrete 
8. origin: multi-valued discrete 

D'après la question précédente, le poids, l'année du modèle et le biais sont des facteurs important pour le calcul de la consommation... Jusqu'ici, nous n'avons pas pris en compte l'origine qui était difficile à coder.

### Encodage de l'origine

La variable origine est accessible de la manière suivante:

```
  origine = data.values[:,-2]
```
Il faut le faire au début du traitement pour bien conserver la séparation en l'apprentissage et le test.

Au moins les deux derniers facteurs discrets pourraient être traités différemment en one-hot encoding:
$$X_j = x \in \{1, \ldots, K\} \Rightarrow [0, 0, 1, 0] \in \{0, 1\}^K$$

La valeur $x$ donne l'index de la colonne non nulle.

### Encodage de l'année

Pour l'année, il est possible de procéder de la même manière, mais il préférable de découper les années en 10 catégories puis d'encoder pour limiter le nombre de dimensions.